<a href="https://colab.research.google.com/github/Doongwoooo/Dacon/blob/master/Sea%20ice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, glob
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, Input
from keras.models import Sequential
import numpy as np
import pylab as plt

import re
patten_date = re.compile('\d{6}')

from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Conv3D, ConvLSTM2D, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%tensorflow_version 2.x
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.4.1
Running on TPU  ['10.110.162.82:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.110.162.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.110.162.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
def load_npy(path):
    npy = np.load(path)
    return npy

def show_data(npy):
    num_channel = npy.shape[-1]
    plt.figure(figsize=(3*num_channel, 5))
    for channel in range(num_channel):
        tmpimg = npy[:, :, channel]
        plt.subplot(1, num_channel, channel+1)
        plt.imshow(tmpimg)
    plt.tight_layout()
    plt.show()
    plt.close()

def show_one_data(npy):
    num_channel = npy.shape[-1]
    plt.figure(figsize=(3, 5))
    tmpimg = npy[:, :]
    plt.imshow(tmpimg)
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
path = '/content/drive/MyDrive/Dacon/train/'
folder = list(os.listdir(path))
train_path = sorted(glob.glob(path+'train/*.npy'))
train_path = train_path[-30*12:]

In [ ]:
X_map = []
X_mon = []
for file in folder:
    npy = (load_npy(path + file)[:, :, 0] + load_npy(path + file)[:, :, 1])/250
    X_map.append(np.array(npy))
    date = str(patten_date.match(file))
    X_mon.append(date[-2:])
X_map = np.array(X_map)

In [ ]:
X_map.shape

(494, 448, 304)

In [ ]:
window_size = 12 * 2 # 몇년?

In [ ]:
x_train = []
y_train = []

for i in range(X_map.shape[0]-window_size):
    x_train.append(X_map[i:i+window_size, :, :])
    y_train.append(X_map[i+window_size:i+window_size+1, :, :])
    
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
stride = 64
feature_size = 128
x_train_ = []
y_train_ = []
for y in range(x_train.shape[0]):
    for i in range((x_train.shape[2]-feature_size)//stride+1):
        for j in range((x_train.shape[3]-feature_size)//stride+1):
            x_ = x_train[y, :, stride*i:stride*i+feature_size, stride*j:stride*j+feature_size]
            y_ = y_train[y, :, stride*i:stride*i+feature_size, stride*j:stride*j+feature_size]
            x_train_.append(x_)
            y_train_.append(y_)

x_train_ = np.array(x_train_)
y_train_ = np.array(y_train_)

x_val_ = []
y_val_ = []

for i in range((x_train.shape[2]-feature_size)//stride+1):
    for j in range((x_train.shape[3]-feature_size)//stride+1):
        x_ = x_train[-1, :, stride*i:stride*i+feature_size, stride*j:stride*j+feature_size]
        y_ = y_train[-1, :, stride*i:stride*i+feature_size, stride*j:stride*j+feature_size]
        x_val_.append(x_)
        y_val_.append(y_)
            
x_val_ = np.array(x_val_)
y_val_ = np.array(y_val_)

In [ ]:
x_train_.shape, y_train_.shape, x_val_.shape, y_val_.shape

In [ ]:
with strategy.scope():
    ice_input = Input((None, x_train_.shape[2], x_train_.shape[3]))

    convLSTM = ConvLSTM2D(filters=64, kernel_size=3, padding='same', return_sequences=True)(ice_input)
    convLSTM = BatchNormalization()(convLSTM)

    convLSTM = ConvLSTM2D(filters=64, kernel_size=3, padding='same', return_sequences=True)(convLSTM)
    convLSTM = BatchNormalization()(convLSTM)

    convLSTM = ConvLSTM2D(filters=64, kernel_size=3, padding='same', return_sequences=True)(convLSTM)
    convLSTM = BatchNormalization()(convLSTM)

    month_input = Input((None))
    month_layer = layers.Dense(1)

    concat = layers.concatenate([convLSTM, month_layer], axis = -1)
    conca_layer = layers.dense((samples.shape[-1]), activation = 'relu')(concat)

    outputs = Conv3D(filters=1, kernel_size=3, activation='relu', padding='same', data_format='channels_last')(conca_layer)

    model = Model(inputs, outputs)
    model.compile(loss='mae', optimizer=Adam())
    
model.summary()

In [ ]:
if not (os.path.isdir('models')):
    os.makedirs('models')

callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath = './models/convlstm_model.h5',
        monitor='val_loss',
        save_best_only=True
    )
]

hist = model.fit(train_dataset, epochs=20, validation_data=val_dataset, callbacks=callbacks_list) #두 시간 가량 소요

In [ ]:
plt.plot(hist.history['loss'][1:])
plt.plot(hist.history['val_loss'][1:])
plt.show()

In [ ]:
model = load_model('./models/convlstm_model.h5')

In [ ]:
pred = model.predict(next(iter(val_dataset))[0])
pred = np.where(pred>250, 250, pred)

In [ ]:
for i in range(BATCH_SIZE):
    plt.subplot(141)
    plt.imshow(next(iter(val_dataset))[1][i,0,:,:,0])
    plt.subplot(142)
    plt.imshow(pred[i,0,:,:,0])
    plt.show()
    print(np.mean(np.abs(next(iter(val_dataset))[1][i,0,:,:,0] - pred[i,0,:,:,0])))

In [ ]:
x_test = train_month[:,-1*window_size:,:,:,:1]
x_test = np.swapaxes(x_test , 0, 1)

In [ ]:
x_test_ = []
for m in range(12):
    feature = []
    for i in range((x_train.shape[3]-feature_size)//stride+1):
        for j in range((x_train.shape[4]-feature_size)//stride+1):
            x_ = x_test[:, m, stride*i:stride*i+feature_size, stride*j:stride*j+feature_size, :1]/250
            feature.append(x_)
    feature = np.array(feature)
    x_test_.append(feature)
x_test_ = np.array(x_test_)

In [ ]:
x_test_.shape

In [ ]:
preds = []
for m in tqdm(range(12)):
    pred = model.predict(x_test_[m])
    pred = np.where(pred>250, 250, pred)
    preds.append(pred)

In [ ]:
voting_mask = np.zeros((x_train.shape[-3], x_train.shape[-2]))
ones = np.ones((feature_size, feature_size))

for i in range((x_train.shape[3]-feature_size)//stride+1):
    for j in range((x_train.shape[4]-feature_size)//stride+1):
        voting_mask[stride*i:stride*i+feature_size, stride*j:stride*j+feature_size] += ones

In [ ]:
plt.imshow(voting_mask)

In [ ]:
results = []
for m in range(12):
    pred = np.zeros((x_train.shape[-3], x_train.shape[-2]))
    k = 0
    for i in range((x_train.shape[3]-feature_size)//stride+1):
        for j in range((x_train.shape[4]-feature_size)//stride+1):
            pred[stride*i:stride*i+feature_size, stride*j:stride*j+feature_size] += preds[m][k,-1,:,:,0]
            k+=1
    pred /= voting_mask
    results.append(pred[:448, :304])
results = np.array(results)

In [ ]:
for m in range(12):
    plt.imshow(results[m])
    plt.show()
    print(m+1, '\n')

In [ ]:
submission = pd.read_csv(path+'sample_submission.csv')
submission

In [ ]:
sub_2019 = submission.loc[:11, ['month']].copy()
sub_2020 = submission.loc[12:].copy()

sub_2019 = pd.concat([sub_2019, (pd.DataFrame(results.reshape([12,-1])))], axis=1)
sub_2019.columns = submission.columns
submission = pd.concat([sub_2019, sub_2020])

In [ ]:
submission

In [ ]:
submission.to_csv(path+'submission.csv', index=False)